# 2024 Day 13

https://adventofcode.com/2024/day/13

https://adventofcode.com/2024/day/13/input

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
test = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

In [3]:
text = open("input-13.txt").read().strip()
# text

## Line intersection

Trying to minimize room for error, loosing track of whether intersection occurs at integer-valued coordinates...

In [4]:
import sympy as sy

In [5]:
y1 = sy.simplify(sy.Subs('m_1*x+b_1', ('m_1', 'b_1'), ('y_d1/x_d1', 'y_p1 - (y_d1/x_d1) * x_p1')))
y1

x*y_d1/x_d1 + y_p1 - x_p1*y_d1/x_d1

In [6]:
print(y1)

x*y_d1/x_d1 + y_p1 - x_p1*y_d1/x_d1


In [7]:
y2 = sy.sympify(str(y1).replace('1', '2'))
y2

x*y_d2/x_d2 + y_p2 - x_p2*y_d2/x_d2

In [8]:
solution = sy.solve(sy.Eq(y1, y2), 'x')[0]
solution

(x_d1*x_d2*y_p1 - x_d1*x_d2*y_p2 + x_d1*x_p2*y_d2 - x_d2*x_p1*y_d1)/(x_d1*y_d2 - x_d2*y_d1)

In [9]:
print(solution)

(x_d1*x_d2*y_p1 - x_d1*x_d2*y_p2 + x_d1*x_p2*y_d2 - x_d2*x_p1*y_d1)/(x_d1*y_d2 - x_d2*y_d1)


In [10]:
final_y = sy.simplify(sy.simplify(sy.Subs(y1, ('x',), solution)))
final_y

(x_d1*y_d2*y_p1 - x_d2*y_d1*y_p2 - x_p1*y_d1*y_d2 + x_p2*y_d1*y_d2)/(x_d1*y_d2 - x_d2*y_d1)

In [11]:
print(final_y)

(x_d1*y_d2*y_p1 - x_d2*y_d1*y_p2 - x_p1*y_d1*y_d2 + x_p2*y_d1*y_d2)/(x_d1*y_d2 - x_d2*y_d1)


## Implementation

In [17]:
button_regex = re.compile(r'Button [AB]: X\+(\d+), Y\+(\d+)')
prize_regex = re.compile(r'Prize: X=(\d+), Y=(\d+)')

class Vec2:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Vec2(x={self.x}, y={self.y})'
        
    def __add__(self, rhs):
        return self.__class__(self.x + rhs.x, self.y + rhs.y)
    
    def __sub__(self, rhs):
        return self.__class__(self.x - rhs.x, self.y - rhs.y)
    
    def __mul__(self, rhs):
        Self = self.__class__
        if isinstance(rhs, Self):
            return self.x * rhs.x + self.y * rhs.y
        else:
            return Self(self.x * rhs, self.y * rhs)
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
        
    def __rmul__(self, lhs):
        Self = self.__class__
        if isinstance(lhs, Self):
            return self.x * lhs.x + self.y * lhs.y
        else:
            return Self(self.x * lhs, self.y * lhs)
    
    def is_int(self):
        return self.x % 1 < 1e-3 and self.y % 1 < 1e-3
    
class Line:
    def __init__(self, point, direction):
        # y - y0 = m * (x - x0)
        # y = m * x - m * x0 + y0
        self.point = point
        self.direction = direction
        self.m = direction.y / direction.x
        self.b = point.y - self.m * point.x
        
    def __call__(self, x):
        return self.m * x + self.b
        
    def __repr__(self):
        return f'Line({self.m}x + {self.b})'
        
    def intersect(self, other):
        x_d1 = self.direction.x
        x_d2 = other.direction.x
        y_d1 = self.direction.y
        y_d2 = other.direction.y
        
        x_p1 = self.point.x
        x_p2 = other.point.x
        y_p1 = self.point.y
        y_p2 = other.point.y
        
        x = (x_d1*x_d2*y_p1 - x_d1*x_d2*y_p2 + x_d1*x_p2*y_d2 - x_d2*x_p1*y_d1)/(x_d1*y_d2 - x_d2*y_d1)
        x, xrem = divmod(x_d1*x_d2*y_p1 - x_d1*x_d2*y_p2 + x_d1*x_p2*y_d2 - x_d2*x_p1*y_d1, x_d1*y_d2 - x_d2*y_d1)
        y = (x_d1*y_d2*y_p1 - x_d2*y_d1*y_p2 - x_p1*y_d1*y_d2 + x_p2*y_d1*y_d2)/(x_d1*y_d2 - x_d2*y_d1)
        y, yrem = divmod(x_d1*y_d2*y_p1 - x_d2*y_d1*y_p2 - x_p1*y_d1*y_d2 + x_p2*y_d1*y_d2, x_d1*y_d2 - x_d2*y_d1)
        if xrem == 0 and yrem == 0:
            return Vec2(x, y)
        else:
            return None

class Game:
    def __init__(self, para, bonus=0):
        lines = para.strip().split('\n')
        self.a = Vec2(*map(int, button_regex.match(lines[0]).groups()))
        self.b = Vec2(*map(int, button_regex.match(lines[1]).groups()))
        self.cost = Vec2(3, 1)
        self.origin = Vec2(0, 0)
        gx, gy = map(int, prize_regex.match(lines[2]).groups())
        self.goal = Vec2(gx + bonus, gy + bonus)
        
    def intersection(self):
        cross = Line(self.origin, self.b).intersect(Line(self.goal, self.a))
        if cross is not None and 0 <= cross.x <= self.goal.x:
            return cross
        else:
            return None
        
    def play(self):
        cross = self.intersection()
        
        if cross is not None:
            x = (self.goal.x - cross.x) / self.a.x
            y = (cross.x / self.b.x)
            if x % 1 == 0 and y % 1 == 0:
                steps = Vec2(x, y)
                return int(steps * self.cost)
            
        return 0
        
    def __repr__(self):
        return f'Game(a={self.a}, b={self.b}, goal={self.goal})'

## Part 1

In [18]:
test_games = list(map(Game, test.split('\n\n')))
sum(game.play() for game in test_games)

480

In [19]:
games = list(map(Game, text.split('\n\n')))
sum(game.play() for game in games)

33921

## Part 2

In [20]:
test_games = [Game(para, bonus=10000000000000) for para in test.split('\n\n')]
sum(game.play() for game in test_games)

875318608908

In [21]:
games = [Game(para, bonus=10000000000000) for para in text.split('\n\n')]
sum(game.play() for game in games)

82261957837868